In [ ]:
def source_target_to_adj_matrix(arr):
    key_map = {}
    x_arr = []
    for idx, elem in enumerate(arr):
        if elem['source'] not in x_arr and elem['source'] not in key_map:
            key_map[elem['source']] = len(x_arr)
            x_arr.append(elem['source'])
        if elem['target'] not in x_arr and elem['target'] not in key_map:
            key_map[elem['target']] = len(x_arr)
            x_arr.append(elem['target'])
            
    x, y = len(list(set(x_arr))), len(list(set(x_arr)))
    adj_matrix = np.zeros(shape=(x, y))
    
    for idx, elem in enumerate(arr):
        if(elem['source'] != -1):
            source_elem_pos = key_map[elem['source']]
        if(elem['target'] != -1):
            target_elem_pos = key_map[elem['target']]
        adj_matrix[source_elem_pos][target_elem_pos] = elem['weight']
    return adj_matrix, key_map

In [ ]:
def module_topology(state, module):
    g = state.g
    df = state.df
    topology = nx.Graph()
    nodes = df.loc[df['module'] == module]['name'].unique().tolist()
    source_target_data = []
    entry_node = nodes[0]
    for idx, node in enumerate(nodes):
        hierarchy_pd = pd.DataFrame(columns=['source', 'target', 'weight', 'level', 'type'])
        print(g)
        neighbors = sorted(g[node].items(), key=lambda edge: edge[1]['weight'])
        print(neighbors)
        for idx, n in enumerate(neighbors):
            source_node = node
            target_node = n[0]
            weight = n[1]['weight']
            level = idx
            curr_module = state.df[state.df['name'] == n[0]]['module'].unique()[0]
            #print(n[0], curr_module)
            if(curr_module != module):
                type_node = 'exit'
                level = -1
                #print('{0} is an exit node'.format(n[0]))
            else:
                type_node = 'normal'
            source_target_data.append({
                    "source": source_node,
                    "target": target_node,
                    "weight": weight,
                    "level": level,
                    "type": type_node
            })
    topology_adj_matrix, topology_key_map = source_target_to_adj_matrix(source_target_data)
    #hierarchy_adj_matrix[1][1] = 0.0
    #print(nx.find_cycle(state.hg, orientation="original"))
    #print(nx.is_tree(state.hg))
    topology_df = nx.from_numpy_matrix(topology_adj_matrix)
    
    return topology_df, topology_key_map

In [ ]:
def draw_hierarchy(state):
    pos = hierarchy_pos(state.hierarchy, 1)
    nx.draw(state.bfs_tree, pos=pos, with_labels=True)
    plt.show()
    return source_target_data

In [ ]:
def verify_topology(states):
#     module = 'libmonitor.so.0.0.0'
    module = 'Unkno'
    for idx, state in enumerate(states):
        state.hierarchy, keymap = module_topology(state, module)
        print(state.hierarchy.nodes())
        print(state.hierarchy.edges())
        for idx, key in enumerate(keymap):
            print(key, keymap[key], state.df[state.df['name'] == key]['module'].tolist()[0])

verify_topology(states)